<a href="https://colab.research.google.com/github/SONG-0502/1st-pap/blob/main/%E5%8D%8F%E5%90%8C%E8%BF%87%E6%BB%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

电影推荐系统 - 使用“ml-100k”数据集实现协同过滤，并生成pkl文件并评估
我们将使用 MovieLens 100K 数据集 来实现一个基于协同过滤的电影推荐系统。整个过程分为以下几个步骤：

加载和预处理数据：加载 MovieLens 100K 数据集（包含评分、电影信息等），并构建用户-电影评分矩阵。
实现协同过滤：使用用户-电影评分矩阵计算电影之间的相似度（物品协同过滤）。
推荐系统实现：通过计算电影之间的相似度来推荐相似的电影。
模型保存：使用 pickle 将协同过滤的推荐模型和相似度矩阵保存为 .pkl 文件。
评估推荐系统：通过精确度（Precision）、召回率（Recall）等评估推荐系统的效果。
1. 数据集概述
数据文件：
u.data：用户对电影的评分数据，包含字段：userId，movieId，rating，timestamp。
u.item：电影的详细信息，包含字段：movieId，title，release date，video release date等。
u.user：用户的详细信息，包含字段：userId，age，gender，occupation，zip code等。
我们将使用 u.data 和 u.item 来构建推荐系统。



In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle

# 1. 数据加载与预处理

# 读取MovieLens 100k 数据集文件
rating_data_path = "ml-100k/u.data"
movie_data_path = "ml-100k/u.item"

# 读取评分数据
columns = ['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_csv(rating_data_path, sep='\t', names=columns, encoding='latin-1')

# 读取电影信息
movie_columns = ['movieId', 'title']
movies = pd.read_csv(movie_data_path, sep='|', names=movie_columns, encoding='latin-1', usecols=[0, 1])

# 查看数据基本信息
print(f"评分数据行数: {ratings.shape[0]}")
print(f"电影数据行数: {movies.shape[0]}")

# 2. 构建用户-电影评分矩阵
movie_user_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# 3. 计算电影之间的余弦相似度（基于物品的协同过滤）
movie_user_matrix_filled = movie_user_matrix.fillna(0)  # 用0填充空值

# 计算余弦相似度矩阵，注意这里是计算电影-电影相似度
cosine_sim = cosine_similarity(movie_user_matrix_filled.T)

# 4. 保存模型和相似度矩阵
movie_titles = movies.set_index('movieId')['title'].to_dict()
pickle.dump(movie_titles, open("movie_titles.pkl", "wb"))
pickle.dump(cosine_sim, open("cosine_sim.pkl", "wb"))

# 5. 推荐函数（基于物品协同过滤）
def recommend_movie(movie_title, top_n=10):
    """
    根据电影名称推荐相似的电影
    """
    # 获取电影ID
    movie_idx = movies[movies['title'].str.lower() == movie_title.lower()].index[0]

    # 获取该电影的相似度分数
    sim_scores = list(enumerate(cosine_sim[movie_idx]))

    # 排序并获取最相似的前N个电影
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # 排除自己

    # 获取推荐电影标题
    # Add 1 to the movie index to align with movie_titles keys
    recommended_movies = [movie_titles[i[0] + 1] for i in sim_scores]
    return recommended_movies

# 测试推荐系统
movie_name = "Star Wars (1977)"
recommended_movies = recommend_movie(movie_name, top_n=10)
print(f"根据电影《{movie_name}》推荐的10部电影：")
for idx, movie in enumerate(recommended_movies, 1):
    print(f"{idx}. {movie}")

# 6. 推荐系统的评估
# 我们使用均方根误差（RMSE）来评估推荐系统的效果

# 获取所有用户对电影的评分
ratings = pd.read_csv(rating_data_path, sep='\t', names=columns, encoding='latin-1')

# 创建一个评分矩阵（用户-电影矩阵）
ratings_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')

# 计算预测评分
# Transpose ratings_matrix to align dimensions for dot product
predicted_ratings = cosine_sim.dot(ratings_matrix.fillna(0).T) / np.abs(cosine_sim).sum(axis=1)[:, np.newaxis]

# 将预测评分矩阵转换为DataFrame
# Transpose predicted_ratings back to original orientation
predicted_ratings_df = pd.DataFrame(predicted_ratings.T, columns=ratings_matrix.columns, index=ratings_matrix.index)

# 计算均方根误差（RMSE）
# 真实评分
true_ratings = ratings_matrix.values
# 预测评分
predicted_ratings = predicted_ratings_df.values

# 计算RMSE
mse = mean_squared_error(true_ratings[true_ratings > 0], predicted_ratings[true_ratings > 0])
rmse = sqrt(mse)
print(f"推荐系统的RMSE: {rmse}")


评分数据行数: 100000
电影数据行数: 1682
根据电影《Star Wars (1977)》推荐的10部电影：
1. Return of the Jedi (1983)
2. Raiders of the Lost Ark (1981)
3. Empire Strikes Back, The (1980)
4. Toy Story (1995)
5. Godfather, The (1972)
6. Independence Day (ID4) (1996)
7. Indiana Jones and the Last Crusade (1989)
8. Fargo (1996)
9. Silence of the Lambs, The (1991)
10. Star Trek: First Contact (1996)
推荐系统的RMSE: 2.950193296897328
